## EfficientDet
- paper : (2019) Scalable and Efficient Object Detection (Google brain)

- BiFPN과 Model Scaling을 적용하여 COCO 데이터셋에서 가장 높은 정확도 달성
- 연산량, 연산 속도 관점에서 기존 연구 대비 굉장히 효율적인 모델
- Weighted Feature Fusion에서 Fast normalized fusion를 사용하여 가중치를 합침
- Backbone, feature fusion, class/box network에 대한 Compound Scailing 진행

<hr>


## Introduction & Abstract

### 목표 1. Efficient multi-scale feature fusion
- Feature Pyramid Network(FPN), One-Stage Detector(RetinaNet, M2Det), Neural Architecture Search + FPN (NAS-FPN) 등의 선행연구
- 기존의 모두 서로 다른 Input Feature들을 합칠 때 구분없이 단순히 더하는 방식

- 다른 input feature들은 해상도가 다르기 때문에 output feature에 기여하는 정도를 다르게 가져가야 함

- weighted bi-directional FPN(BiFPN) 구조를 제안하여 서로 다른 input feature들의 중요성을 학습

### 목표 2. Model scaling

![efficientdet_result](img/efficientdet_result.png)


## BiFPN

![efficientdet_bifpn](img/efficientdet_bifpn.png)

- (a) 전통적인 FPN 구조
- (b) PANet은 bottom-up Flow를 FPN에 추가
- (c) AutoML의 Neural Architecture Search를 FPN 구조에 적용 불규칙적인 FPN 구조를 보이며 알아보기 힘든 것이 특징
- (a, b) 구조는 같은 scale에서 connection, (c) 구조는 scale이 다른 경우도 connection이 존재하는 Cross-Scale Connection 적용
- (d, e)는 본 논문에서 추가로 제안하고 실험을 한 방식
- (f)는 본 논문에서 최종 제안하는 BiFPN 구조를 의미
- (e)Simplified PANet 방식은 PANet에서 input edge가 1개인 node들은 기여도가 적을 것이라 생각하며 제거
- (f)첫째 node에서 뒤 edge로 path를 추가하여 더 많은 feature들이 fusion되도록 구성
- 본 논문에서는 Bottom-up, Top-down 방식을 여러 번 반복하여 사용 -> 더 high-level한 feature fusion을 할 수 있음

### BIFPN Result

![efficientdet_bifpn_result](img/efficientdet_bifpn_result.png)
- Backbone(EfficientNet-B3)에서 FPN을 BiFPN으로 바꿨을 때의 성능을 측정
    - mAP는 약 4.1 증가하였고 parameter 수와 FLOPS도 적게 사용

### Weighted Feature Fusion
- Features 합칠 때 가중치를 합치는 법

- Unbounded fusion
    - unbounded 되어있기 때문에 학습에 불안정성을 유발    
- SoftMax-based fusion
    - GPU 하드웨어에서 slowdown을 유발

- (본 논문)Fast normalized fusion 방식을 제안
    - weight들은 ReLU를 거치기 때문에 non-zero임이 보장
    - 분모가 0이 되는 것을 막기 위해 0.0001 크기의 입실론
    - Weight 값이 0~1사이로 normalize가 되는 것은 SoftMax와 유사하며
    - SoftMax-based fusion 방식보다 좋은 성능을 보여줌
    
- weight는 scalar(per-feature)로 줄 수 있고, vector (per-channel)로 줄 수 있고 multi-dimensional tensor (per-pixel)로 줄 수 있는데, 본 논문에서는 scalar를 사용하는 것이 정확도와 연산량 측면에서 효율적임을 실험을 통해 밝혔고, scalar weight를 사용

![efficientdet_weighted_feature_fusion](img/efficientdet_weighted_feature_fusion.png)

<hr>

### SoftMax fusion vs Fast Fusion을 비교 결과 
- Fast Fusion을 사용하면 약간의 mAP 하락은 있지만 약 30%의 속도 향상을 달성

![efficientdet_weighted_feature_fusion_result](img/efficientdet_weighted_feature_fusion_result.png)

<hr>


## EfficientDet Architecture

![efficientdet_architecture](img/efficientdet_architecture.png)

- EfficientDet의 backbone
    - ImageNet-pretrained EfficientNet을 사용
- BiFPN
    - top-down, bottom-up bidirectional feature fusion을 반복 사용
    - Feature Network로 사용하였고, level 3-7 feature에 적용

## Compound Scaling

- backbone network에는 EfficientNet-B0 부터 B6까지 사용
- Input의 resolution
    - [512,640,768,896,1024,1280,1408,1536]
    
- backbone network의 크기
    - [B0, B1, B2, B3, B4, B5, B6, B7]
    
- BiFPN
    - [채널수, BiFPN 갯수]
    - [64,2], [88,3],[112,4],[160,5], [224,6], [288, 7], [384, 8]

- Box/class network (layer)
    - 3~5

![efficientdet_compound_scailing](img/efficientdet_compound_scailing.png)

<hr>

## 실험결과

![efficientdet_experiment_result](img/efficientdet_experiment_result.png)


![efficientdet_experiment_result_coco](img/efficientdet_experiment_result_coco.png)
- COCO 데이터셋에서 가장 높은 mAP를 달성

<hr>

## 참고문헌
- https://hoya012.github.io/blog/EfficientDet-Review/